In [1]:
import os
import numpy as np
import scipy.stats as stats
import sklearn.metrics as metrics

In [ ]:
f = open('../outdir.txt', 'r')
outdir = f.read().rstrip('\n')
f = open('../experiment_folder.txt', 'r')
experiment_folder = f.read().rstrip('\n')

##Read encoded vectors file and ordered medical record numbers
with open(experiment_folder + '/encoded_vect.csv') as f:
    rd = csv.reader(f)
    encoded_vect = []
    for r in rd:
        encoded_vect.append(list(map(float, r)))       
with open(experiment_folder + '/mrns.csv') as f:
    rd = csv.reader(f)
    mrns = []
    for r in rd:
        mrns.append(r[0])

# ##Read encoded vectors file and ordered medical record numbers
# with open(experiment_folder + '/LSTMencoded_vect.csv') as f:
#     rd = csv.reader(f)
#     lstm_encoded_vect = []
#     for r in rd:
#         lstm_encoded_vect.append(list(map(float, r)))
        
# with open(experiment_folder + '/LSTMmrns.csv') as f:
#     rd = csv.reader(f)
#     lstm_mrns = []
#     for r in rd:
#         lstm_mrns.append(r[0])

with open(outdir + '/cohort-new_ehr.csv') as f:
    rd = csv.reader(f)
    raw_ehr = {}
    for r in rd:
        raw_ehr.setdefault(r[0], list()).extend(list(map(int, r[1::])))
   
##read the list of diseases associated to the mrns
with open(outdir + '/cohort-mrn_diseases.csv') as f:
    rd = csv.reader(f)
    mrn_disease = {}
    for r in rd:
        mrn_disease[r[0]] = r[1::]
##create a list that orders the first diagnoses according 
##to the list of mrns
mrn_encoded_disease = []
for m in mrns:
    mrn_encoded_disease.append(mrn_disease[m][0])

#Vocabulary dictionary
with open(outdir + '/cohort-new_vocab.csv') as f:
    rd = csv.reader(f)
    next(rd)
    new_vocab = {}
    for r in rd:
        new_vocab[r[1]] = r[0]
        
len_vocab = len(new_vocab)

with open(os.path.join(experiment_folder, "subgroups.csv"), 'r') as f:
    rd = csv.reader(f)
    models = next(rd) ##model labels "cnn+ae", "lstm", "tfidf", "count"
    cluster = {}
    for r in rd:
        cluster.setdefault(r[0], list()).extend(r[1::])
        
with open(os.path.join(experiment_folder, "svd_mat.csv"), 'r') as f:
    rd = csv.reader(f, delimiter=',')
    svd_mat = [r for r in rd]
    
with open(os.path.join(experiment_folder, "raw_data_scaled.csv"), 'r') as f:
    rd = csv.reader(f, delimiter=',')
    raw_data_scaled = [ r for r in rd]
    
with open(os.path.join(experiment_folder, "cluster.csv"), 'r') as f:
    rd = csv.reader(f, delimiter=',')
    cluster.dict.fromkeys(next(rd))
    tmp_mat = [r for r in rd]
    tmp_mat = np.array(tmp_mat).T.tolist()
    for m, v in zip(cluster, tmp_mat):
        cluster[m] = v

## Similarity measure within diseases

In [ ]:
def disease_sim(model_mat, disease_vect, model):
    for d in set(disease_vect):
        tmp_mat = []
        result = []
        for idx, lab in disease_vect:
            if lab == d:
                tmp_mat.append(ehrs_mat[idx])
        pair_sim = 1 - metrics.pairwise.euclidean_distance(tmp_mat)
        mu = np.mean(pair_dist)
        sd = np.sd(pair_dist)
        string = "Model {0} -- Disease {1}:: similarity ({2}, {3})".format(model, d, mu, sd)
        print(string)
        result += string
    f.open(os.path.join(experiment_folder, "similarity_" + model + ".txt"), "w")
    f.writelines([r for r in result])
    f.close()    

## Chi-squared test between new clusters

In [ ]:
def chi_test(raw_ehr, vocab, new_classes, model):
    count_mat = np.zeros(len_vocab, len(set(class_label)))
    for c in set(class_label):
        for idx, m in enumerate(mrns):
            if class_label[idx] == c:
                for tag in raw_data[mrn]:
                    count_mat[tag][c] += 1
    chi2_stat, p_val, dof, ex = stats.chi2_contingency(count_mat)
    string = "Chi-squared test statistics:\
              chi2_stat = {0} -- p_val = {1} -- dof = {2}".format(chi2_stat,
                                                                  p_val,
                                                                  dof)#row = classes, columns = vocab
    print(string)
    f.open(os.path.join(experiment_folder), "chi2_" + model + ".txt", "w")
    f.write(string)
    f.close()

## Run tests on data

#### CNN + AE

In [ ]:
model = "cnn+ae"
disease_vect = [mrn_disease[m][0] for m in mrns]
disease_sim(encoded_vect, disease_vect, model)
chi_test(raw_ehr, new_vocab, cluster[model], model)

#### LSTM

In [ ]:
model = "lstm"
disease_vect = [mrn_disease[m][0] for m in lstm_mrns]
disease_sim(lstm_encoded_vect, disease_vect, model)
chi_test(raw_eh, new_vocab, cluster[model], model)

#### TFIDF

In [ ]:
model = "tfidf"
disease_vect = [mrn_disease[m][0] for m in raw_ehr]
disease_sim(svd_mat, disease_vect, model)
chi_test(raw_ehr, new_vocab, cluster[model], model)

#### COUNT

In [ ]:
model = "count"
disease_vect = [mrn_disease[m][0] for m in raw_ehr]
disease_sim(raw_data_scaled, disease_vect, model)
chi_test(raw_ehr, new_vocab, cluster[model], model)